### Video Game Data Analysis

In [11]:
import pandas as pd
import os
import numpy as np

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(r'C:\Users\akibo\Desktop\Columbia Bootcamp\Pandas-Purchasing\Resources\purchase_data.csv',sep=",")

## Player Count

In [12]:
# You can get the total purchases by leveraging the shape without the header row by subtracting 1

total_purchases=purchase_data.shape[0]-1
print(str(total_purchases))

779


## Purchasing Analysis

In [18]:
# Define variables and calculations for columns
average_age=purchase_data['Age'].mean()
amt_of_items=purchase_data['Item ID'].nunique()
average_price=purchase_data['Price'].mean()
total_revenue=purchase_data['Price'].sum()

# Set up the column names so that it can be passed into the dataframe
Column_names1=["Average Age","Amount of Items","Amount of Purchases","Average Price","Total Revenue"]
row_values1=[average_age,amt_of_items,total_purchases,average_price,total_revenue]

# The summary table will be the column names, then we can insert the row values in the first row, indexed as 0
Summary_Table=pd.DataFrame(index=None,columns=Column_names1)
Summary_Table.loc[0]=row_values1

# Formatting columns
Summary_Table['Average Age']=Summary_Table['Average Age'].astype(float).map("{:,.0f}".format)
Summary_Table['Amount of Items']=Summary_Table['Amount of Items'].astype(float).map("{:,.0f}".format)
Summary_Table['Amount of Purchases']=Summary_Table['Amount of Purchases'].astype(float).map("{:,.0f}".format)
Summary_Table['Average Price']=Summary_Table['Average Price'].astype(float).map("${:,.2f}".format)
Summary_Table['Total Revenue']=Summary_Table['Total Revenue'].astype(float).map("${:,.2f}".format)

Summary_Table

,Average Age,Amount of Items,Amount of Purchases,Average Price,Total Revenue
0,23,179,779,$3.05,"$2,379.77"


## Gender Demographics and Purchasing Analysis by Gender

In [115]:
# Create a function so that we only have to calculate percentage once (DRY)
def Percentage_Calc(x):
    return (x/total_purchases)*100

# Define variables and calculate the amount of males, females, and "other"
Males=purchase_data[purchase_data.Gender==str('Male')].shape[0]
Females=purchase_data[purchase_data.Gender==str('Female')].shape[0]
Other_Count=purchase_data[purchase_data.Gender==str("Other / Non-Disclosed")].shape[0]

Gender_header=["Gender","Count","Percentage"]
Gender_list=["Males","Females","Other"]

# We can use a dictionary to pass in the values, and leverage the function to get percentage
Gender_dict={"Gender":["Males","Females","Other"],"Count":[Males,Females,Other_Count],"Percentage":[Percentage_Calc(Males),Percentage_Calc(Females),Percentage_Calc(Other_Count)]}

# Use the dictionary created to make a new dataframe
Gender_Summary=pd.DataFrame(data=Gender_dict)

# Format percentage column
Gender_Summary['Percentage']=Gender_Summary['Percentage'].astype(float).map("{:,.1f} %".format)

Gender_Summary

,Gender,Count,Percentage
0,Males,652,83.7 %
1,Females,113,14.5 %
2,Other,15,1.9 %


## Age Demographics and Purchasing Analysis

In [35]:
# Establish labels for each bin
ages_bins=[0,9,14,19,24,29,34,39]
ages_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39"]

# Establish bins with the pd.cut method
purchase_data["Age Group"]=pd.cut(purchase_data["Age"],ages_bins,labels=ages_labels,include_lowest=False)

# Set up new dataframe with aggregated information
purchase_data_ages=pd.DataFrame(purchase_data.groupby("Age Group").sum().Price)
purchase_data_ages["Count of Purchases"]=purchase_data.groupby("Age Group").count().Price
purchase_data_ages["Average Price"]=purchase_data.groupby("Age Group").mean().Price

# Rename the price column to "Total Purchase Value"
purchase_data_ages=purchase_data_ages.rename(columns={"Price":"Total Purchase Value"})

# Add in column that accounts for the percentage of purchases per group
purchase_data_ages["Percentage Purchased"]=((purchase_data_ages["Total Purchase Value"])/total_revenue)*100

# Format columns
purchase_data_ages['Average Price']=purchase_data_ages['Average Price'].astype(float).map("${:,.2f}".format)
purchase_data_ages['Total Purchase Value']=purchase_data_ages['Total Purchase Value'].astype(float).map("${:,.2f}".format)
purchase_data_ages['Percentage Purchased']=purchase_data_ages['Percentage Purchased'].astype(float).map("{:,.2f} %".format)

purchase_data_ages

,Total Purchase Value,Count of Purchases,Average Price,Percentage Purchased
Age Group,,,,
<10,$77.13,23,$3.35,3.24 %
10-14,$82.78,28,$2.96,3.48 %
15-19,$412.89,136,$3.04,17.35 %
20-24,"$1,114.06",365,$3.05,46.81 %
25-29,$293.00,101,$2.90,12.31 %
30-34,$214.00,73,$2.93,8.99 %
35-39,$147.67,41,$3.60,6.21 %


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [41]:
# Set up purchasing data organized by "SN", or user name
purchase_data_users=pd.DataFrame(purchase_data.groupby("SN").sum().Price)

# Rename the column so it denotes its true value
purchase_data_users=purchase_data_users.rename(columns={"Price":"Total Purchase Value"})

# Determine average purchase price and purchase count
purchase_data_users["Average Purchase Price"]=purchase_data.groupby("SN").mean().Price
purchase_data_users["Purchase Count"]=purchase_data.groupby("SN").count().Price

# Print out dataframe sorted by average purchase price in descending order, slicing in order to show the top 5 spenders
purchase_data_users.sort_values("Total Purchase Value",ascending=False)[:5]


,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,18.96,3.792000,5
Idastidru52,15.45,3.862500,4
Chamjask73,13.83,4.610000,3
Iral74,13.62,3.405000,4
Iskadarya95,13.10,4.366667,3


## Most Popular and Profitable Items

In [104]:
# Establish new dataframe with the Item ID
purchase_data_items=pd.DataFrame(purchase_data.groupby("Item ID").sum().Price)
purchase_data_items=purchase_data_items.rename(columns={"Price":"Total Revenue"})

# insert the name of the items at a location next to the respective ID
purchase_data_items.insert(0,"Item Name",purchase_data["Item Name"])

# Insert column for item price and amount sold
purchase_data_items.insert(1,"Item Price",purchase_data.groupby("Item ID").mean().Price)
purchase_data_items["Amount Sold"]=purchase_data.groupby("Item ID").count().Price

# Formatting columns
purchase_data_items['Item Price']=purchase_data_items['Item Price'].astype(float).map("${:,.2f}".format)

# Print new dataframe sorted by highest Amount Sold in descending order, indexing for top 5
purchase_data_items.sort_values('Amount Sold',ascending=False)[0:5]

,Item Name,Item Price,Total Revenue,Amount Sold
Item ID,,,,
92,"Betrayal, Whisper of Grieving Widows",$4.61,59.99,13
178,"Despair, Favor of Due Diligence",$4.23,50.76,12
145,Hopeless Ebon Dualblade,$4.58,41.22,9
132,Fiery Glass Crusader,$3.22,28.99,9
108,Malificent Bag,$3.53,31.77,9


In [102]:
# Print new dataframe sorted by highest total revenue/purchase value in descending order, indexing for top 5
purchase_data_items.sort_values('Total Revenue',ascending=False)[0:5]

,Item Name,Item Price,Total Revenue,Amount Sold
Item ID,,,,
92,"Betrayal, Whisper of Grieving Widows",$4.61,59.99,13
178,"Despair, Favor of Due Diligence",$4.23,50.76,12
82,Azurewrath,$4.90,44.10,9
145,Hopeless Ebon Dualblade,$4.58,41.22,9
103,"Thorn, Satchel of Dark Souls",$4.35,34.80,8
